# 🗃️ Week 9 — Group Session: Introduction to Databases and SQL

```
╔══════════════════════════════════════════════════════════════════╗
║  🤝 COLLABORATIVE DATABASE DESIGN & ADVANCED SQL                 ║
║                                                                  ║
║  "Great databases are designed, not just created."               ║
║                                                                  ║
║  This session covers:                                            ║
║  • JOINs - Connecting related tables                            ║
║  • Aggregations - GROUP BY, SUM, AVG, COUNT                     ║
║  • Transactions - ACID properties                               ║
║  • Schema Design - Normalization basics                         ║
╚══════════════════════════════════════════════════════════════════╝
```

### 🎯 Session Format: Collaborative Problem Solving

| Role | Responsibility |
|------|----------------|
| 👷 **Schema Designer** | Define table structures and relationships |
| ✍️ **Query Author** | Write SQL queries to solve problems |
| ✅ **Verifier** | Test queries and validate results |

> 💡 **Rotate roles** after each activity so everyone practices each skill!

## 📋 Learning Objectives

By the end of this session, you will be able to:

| Objective | Skills |
|-----------|--------|
| 🔗 **Design Related Tables** | Foreign keys, one-to-many relationships |
| 🤝 **Write JOIN Queries** | INNER JOIN, LEFT JOIN, relationship navigation |
| 📊 **Aggregate Data** | GROUP BY, SUM, AVG, COUNT, HAVING |
| 🔒 **Use Transactions** | BEGIN, COMMIT, ROLLBACK, atomicity |
| 🏗️ **Collaborate on Schema** | Normalization, data integrity |

### Prerequisites:
- ✅ Completed Session 1 (Basic CRUD operations)
- ✅ Understanding of CREATE, INSERT, SELECT, UPDATE, DELETE
- ✅ Familiarity with parameterized queries

---

## 🔗 Activity 1: Two Tables and JOINs

```
╔══════════════════════════════════════════════════════════════════╗
║  Understanding Table Relationships                               ║
║                                                                  ║
║  users (1) ────────< (many) orders                              ║
║                                                                  ║
║  • One user can have many orders                                ║
║  • Each order belongs to exactly one user                       ║
║  • Connected via foreign key (user_id)                          ║
╚══════════════════════════════════════════════════════════════════╝
```

### What is a JOIN?

A JOIN combines rows from two or more tables based on a related column:

| JOIN Type | Description | Use Case |
|-----------|-------------|----------|
| **INNER JOIN** | Only matching rows | Show users with orders |
| **LEFT JOIN** | All left + matching right | Show all users (even without orders) |
| **RIGHT JOIN** | All right + matching left | Show all orders (rare) |
| **FULL OUTER JOIN** | All rows from both | Complete picture (not in SQLite) |

### Visual Representation:

```
USERS table:                ORDERS table:
┌────┬───────┐             ┌────┬─────────┬───────┐
│ id │ name  │             │ id │ user_id │ total │
├────┼───────┤             ├────┼─────────┼───────┤
│ 1  │ Ana   │─────┐       │ 1  │ 1       │ 19.99 │
│ 2  │ Bob   │─────┼──────>│ 2  │ 1       │ 5.00  │
│ 3  │ Carol │     │       │ 3  │ 2       │ 12.50 │
└────┴───────┘     └──────>│    │         │       │
                           └────┴─────────┴───────┘
```

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🚫 BAD PRACTICE: JOINs Done Wrong                               ║
# ╚══════════════════════════════════════════════════════════════════╝
# What NOT to do when working with multiple tables

import sqlite3

# ❌ Bad: No foreign key constraint
# ❌ Bad: No data integrity checks
# ❌ Bad: Using string formatting (SQL injection risk)
# ❌ Bad: No error handling
# ❌ Bad: Manual resource management

conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# No foreign key enforcement
cur.execute('CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT)')
cur.execute('CREATE TABLE orders (id INTEGER PRIMARY KEY, user_id INTEGER, total REAL)')

# Insert data - notice no validation that user_id exists!
cur.executemany('INSERT INTO users (name) VALUES (?)', [('Ana',), ('Bob',)])
cur.executemany('INSERT INTO orders (user_id, total) VALUES (?, ?)', [
    (1, 19.99),
    (1, 5.00),
    (2, 12.50),
    (999, 100.00)  # ⚠️ user_id 999 doesn't exist! No error!
])
conn.commit()

# Bad: Using string concatenation for queries
user_name = "Ana"
query = f"SELECT u.name, o.total FROM users u JOIN orders o ON u.id = o.user_id WHERE u.name = '{user_name}'"
cur.execute(query)  # SQL INJECTION VULNERABILITY!
print("❌ Bad Practice Results:")
print(cur.fetchall())

# Orphan order exists with no user!
cur.execute('SELECT * FROM orders WHERE user_id = 999')
print(f"⚠️ Orphan order: {cur.fetchall()}")

conn.close()  # Manual cleanup

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🌱 NOVICE: Basic JOINs with Proper Practices                   ║
# ╚══════════════════════════════════════════════════════════════════╝
# Getting started with table relationships

import sqlite3

# Create connection with foreign key enforcement
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# ✅ Enable foreign key constraints (OFF by default in SQLite!)
cur.execute('PRAGMA foreign_keys = ON')

# ✅ Define tables with foreign key constraint
cur.execute('''
    CREATE TABLE users (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL
    )
''')

cur.execute('''
    CREATE TABLE orders (
        id INTEGER PRIMARY KEY,
        user_id INTEGER NOT NULL,
        total REAL NOT NULL,
        FOREIGN KEY (user_id) REFERENCES users(id)
    )
''')

# Insert users first
cur.executemany('INSERT INTO users (name) VALUES (?)', [
    ('Ana',), ('Bob',), ('Carol',)
])

# Insert orders with valid user_ids
cur.executemany('INSERT INTO orders (user_id, total) VALUES (?, ?)', [
    (1, 19.99),  # Ana's order
    (1, 5.00),   # Ana's second order
    (2, 12.50),  # Bob's order
])
conn.commit()

# ✅ Try inserting orphan order - will fail!
try:
    cur.execute('INSERT INTO orders (user_id, total) VALUES (?, ?)', (999, 100.00))
except sqlite3.IntegrityError as e:
    print(f"✅ Foreign key prevented bad insert: {e}")

# ✅ INNER JOIN - get users with their orders
print("\n🌱 INNER JOIN - Users with Orders:")
cur.execute('''
    SELECT users.name, orders.total 
    FROM users 
    INNER JOIN orders ON users.id = orders.user_id
''')
for row in cur.fetchall():
    print(f"  {row[0]}: ${row[1]:.2f}")

# ✅ LEFT JOIN - get ALL users (even without orders)
print("\n🌱 LEFT JOIN - All Users (Carol has no orders):")
cur.execute('''
    SELECT users.name, orders.total 
    FROM users 
    LEFT JOIN orders ON users.id = orders.user_id
''')
for row in cur.fetchall():
    total = f"${row[1]:.2f}" if row[1] else "No orders"
    print(f"  {row[0]}: {total}")

conn.close()

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🔧 INTERMEDIATE: JOINs with Aggregations                       ║
# ╚══════════════════════════════════════════════════════════════════╝
# Combining related data with SQL aggregate functions

import sqlite3
from typing import List, Tuple

def setup_database() -> sqlite3.Connection:
    """Create and populate a sample e-commerce database."""
    conn = sqlite3.connect(':memory:')
    conn.execute('PRAGMA foreign_keys = ON')
    conn.row_factory = sqlite3.Row  # Named column access
    
    cur = conn.cursor()
    
    # Create schema
    cur.executescript('''
        CREATE TABLE users (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            email TEXT UNIQUE
        );
        
        CREATE TABLE orders (
            id INTEGER PRIMARY KEY,
            user_id INTEGER NOT NULL,
            total REAL NOT NULL,
            order_date TEXT DEFAULT CURRENT_DATE,
            FOREIGN KEY (user_id) REFERENCES users(id)
        );
    ''')
    
    # Insert sample data
    users = [('Ana', 'ana@email.com'), ('Bob', 'bob@email.com'), ('Carol', 'carol@email.com')]
    cur.executemany('INSERT INTO users (name, email) VALUES (?, ?)', users)
    
    orders = [
        (1, 19.99, '2024-01-15'), (1, 5.00, '2024-01-20'),
        (1, 25.50, '2024-02-01'), (2, 12.50, '2024-01-18'),
        (2, 45.00, '2024-02-10')
        # Note: Carol has no orders
    ]
    cur.executemany('INSERT INTO orders (user_id, total, order_date) VALUES (?, ?, ?)', orders)
    conn.commit()
    
    return conn

# ═══════════════════════════════════════════════════════════════════
# 🔧 Aggregation Queries with JOINs
# ═══════════════════════════════════════════════════════════════════

conn = setup_database()
cur = conn.cursor()

# Total spend per user using GROUP BY
print("🔧 Total Spend Per User:")
cur.execute('''
    SELECT 
        u.name,
        COUNT(o.id) as order_count,
        COALESCE(SUM(o.total), 0) as total_spend,
        COALESCE(AVG(o.total), 0) as avg_order
    FROM users u
    LEFT JOIN orders o ON u.id = o.user_id
    GROUP BY u.id
    ORDER BY total_spend DESC
''')

for row in cur.fetchall():
    print(f"  {row['name']}: {row['order_count']} orders, "
          f"${row['total_spend']:.2f} total, ${row['avg_order']:.2f} avg")

# HAVING clause to filter aggregated results
print("\n🔧 High-Value Customers (total > $30):")
cur.execute('''
    SELECT u.name, SUM(o.total) as total_spend
    FROM users u
    INNER JOIN orders o ON u.id = o.user_id
    GROUP BY u.id
    HAVING SUM(o.total) > 30
''')

for row in cur.fetchall():
    print(f"  {row['name']}: ${row['total_spend']:.2f}")

# Orders by month
print("\n🔧 Orders by Month:")
cur.execute('''
    SELECT 
        strftime('%Y-%m', order_date) as month,
        COUNT(*) as order_count,
        SUM(total) as revenue
    FROM orders
    GROUP BY month
    ORDER BY month
''')

for row in cur.fetchall():
    print(f"  {row['month']}: {row['order_count']} orders, ${row['revenue']:.2f} revenue")

conn.close()

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  ✨ PYTHONIC: Enterprise Database Operations                    ║
# ╚══════════════════════════════════════════════════════════════════╝
# Production-ready database patterns with type hints and best practices

import sqlite3
from typing import List, Dict, Optional, Any, NamedTuple
from dataclasses import dataclass
from contextlib import contextmanager
from datetime import date

# ═══════════════════════════════════════════════════════════════════
# ✨ Data Classes for Type Safety
# ═══════════════════════════════════════════════════════════════════

@dataclass
class User:
    """Represents a user in the system."""
    id: Optional[int]
    name: str
    email: str
    
    @classmethod
    def from_row(cls, row: sqlite3.Row) -> 'User':
        return cls(id=row['id'], name=row['name'], email=row['email'])

@dataclass
class Order:
    """Represents an order in the system."""
    id: Optional[int]
    user_id: int
    total: float
    order_date: str

class UserStats(NamedTuple):
    """Statistics for a user."""
    name: str
    email: str
    order_count: int
    total_spend: float
    avg_order: float

# ═══════════════════════════════════════════════════════════════════
# ✨ Database Manager Class
# ═══════════════════════════════════════════════════════════════════

class DatabaseManager:
    """
    Enterprise-grade database manager with connection pooling pattern.
    
    Features:
    - Context manager for automatic cleanup
    - Named tuples for type-safe results
    - Parameterized queries throughout
    - Foreign key enforcement
    """
    
    def __init__(self, db_path: str = ':memory:'):
        self.db_path = db_path
        self._connection: Optional[sqlite3.Connection] = None
    
    @contextmanager
    def get_connection(self):
        """Context manager for database connections."""
        conn = sqlite3.connect(self.db_path)
        conn.execute('PRAGMA foreign_keys = ON')
        conn.row_factory = sqlite3.Row
        try:
            yield conn
            conn.commit()
        except Exception:
            conn.rollback()
            raise
        finally:
            conn.close()
    
    def initialize_schema(self) -> None:
        """Create the database schema."""
        with self.get_connection() as conn:
            conn.executescript('''
                CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY,
                    name TEXT NOT NULL,
                    email TEXT UNIQUE NOT NULL
                );
                
                CREATE TABLE IF NOT EXISTS orders (
                    id INTEGER PRIMARY KEY,
                    user_id INTEGER NOT NULL,
                    total REAL NOT NULL CHECK(total >= 0),
                    order_date TEXT DEFAULT CURRENT_DATE,
                    FOREIGN KEY (user_id) REFERENCES users(id) ON DELETE CASCADE
                );
                
                CREATE INDEX IF NOT EXISTS idx_orders_user ON orders(user_id);
                CREATE INDEX IF NOT EXISTS idx_orders_date ON orders(order_date);
            ''')
    
    def add_user(self, user: User) -> int:
        """Add a user and return their ID."""
        with self.get_connection() as conn:
            cursor = conn.execute(
                'INSERT INTO users (name, email) VALUES (?, ?)',
                (user.name, user.email)
            )
            return cursor.lastrowid
    
    def add_order(self, order: Order) -> int:
        """Add an order and return its ID."""
        with self.get_connection() as conn:
            cursor = conn.execute(
                'INSERT INTO orders (user_id, total, order_date) VALUES (?, ?, ?)',
                (order.user_id, order.total, order.order_date)
            )
            return cursor.lastrowid
    
    def get_user_stats(self) -> List[UserStats]:
        """Get statistics for all users using JOIN and aggregation."""
        with self.get_connection() as conn:
            cursor = conn.execute('''
                SELECT 
                    u.name,
                    u.email,
                    COUNT(o.id) as order_count,
                    COALESCE(SUM(o.total), 0) as total_spend,
                    COALESCE(AVG(o.total), 0) as avg_order
                FROM users u
                LEFT JOIN orders o ON u.id = o.user_id
                GROUP BY u.id
                ORDER BY total_spend DESC
            ''')
            
            return [UserStats(*row) for row in cursor.fetchall()]
    
    def get_high_value_customers(self, threshold: float = 50.0) -> List[UserStats]:
        """Get customers who have spent above a threshold."""
        with self.get_connection() as conn:
            cursor = conn.execute('''
                SELECT 
                    u.name,
                    u.email,
                    COUNT(o.id) as order_count,
                    SUM(o.total) as total_spend,
                    AVG(o.total) as avg_order
                FROM users u
                INNER JOIN orders o ON u.id = o.user_id
                GROUP BY u.id
                HAVING SUM(o.total) >= ?
                ORDER BY total_spend DESC
            ''', (threshold,))
            
            return [UserStats(*row) for row in cursor.fetchall()]
    
    def get_monthly_revenue(self) -> List[Dict[str, Any]]:
        """Get revenue breakdown by month."""
        with self.get_connection() as conn:
            cursor = conn.execute('''
                SELECT 
                    strftime('%Y-%m', order_date) as month,
                    COUNT(*) as order_count,
                    SUM(total) as revenue,
                    AVG(total) as avg_order_value
                FROM orders
                GROUP BY month
                ORDER BY month
            ''')
            
            return [dict(row) for row in cursor.fetchall()]

# ═══════════════════════════════════════════════════════════════════
# ✨ Demonstration
# ═══════════════════════════════════════════════════════════════════

# Initialize database
db = DatabaseManager(':memory:')
db.initialize_schema()

# Add users using dataclasses
users = [
    User(None, 'Ana', 'ana@email.com'),
    User(None, 'Bob', 'bob@email.com'),
    User(None, 'Carol', 'carol@email.com'),
]

user_ids = {user.name: db.add_user(user) for user in users}

# Add orders
orders = [
    Order(None, user_ids['Ana'], 19.99, '2024-01-15'),
    Order(None, user_ids['Ana'], 5.00, '2024-01-20'),
    Order(None, user_ids['Ana'], 75.50, '2024-02-01'),
    Order(None, user_ids['Bob'], 12.50, '2024-01-18'),
    Order(None, user_ids['Bob'], 45.00, '2024-02-10'),
]

for order in orders:
    db.add_order(order)

# Display results
print("✨ PYTHONIC: Enterprise Database Operations\n")

print("📊 All User Statistics:")
print("-" * 60)
for stats in db.get_user_stats():
    print(f"  {stats.name} ({stats.email})")
    print(f"    Orders: {stats.order_count}, Total: ${stats.total_spend:.2f}, "
          f"Avg: ${stats.avg_order:.2f}")

print("\n🌟 High-Value Customers (>$50):")
print("-" * 60)
for stats in db.get_high_value_customers(50.0):
    print(f"  {stats.name}: ${stats.total_spend:.2f}")

print("\n📅 Monthly Revenue:")
print("-" * 60)
for month in db.get_monthly_revenue():
    print(f"  {month['month']}: {month['order_count']} orders, "
          f"${month['revenue']:.2f} revenue")

---

## 🎯 Group Activity 1: Total Spend Per User

### Your Task:

Write a query that computes the **total spend per user** using SQL aggregation.

Requirements:
1. Use a JOIN to connect `users` and `orders` tables
2. Use `GROUP BY` to aggregate per user
3. Use `SUM()` to calculate total spend
4. Order results by total spend (highest first)

### 💡 Hints:

```sql
-- Template:
SELECT 
    u.name,
    SUM(o.total) as total_spend
FROM users u
_______ JOIN orders o ON u.id = o.user_id
GROUP BY _______
ORDER BY _______ DESC
```

### Discussion Questions:
1. Should you use INNER JOIN or LEFT JOIN? What's the difference?
2. What happens to users with no orders?
3. How would you handle NULL values in the total?

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  👩‍💻 YOUR WORKSPACE: Total Spend Per User                        ║
# ╚══════════════════════════════════════════════════════════════════╝
# Complete the query below to calculate total spend per user

import sqlite3

# Setup: Create database with sample data
conn = sqlite3.connect(':memory:')
conn.execute('PRAGMA foreign_keys = ON')
conn.row_factory = sqlite3.Row
cur = conn.cursor()

# Create tables
cur.executescript('''
    CREATE TABLE users (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL
    );
    
    CREATE TABLE orders (
        id INTEGER PRIMARY KEY,
        user_id INTEGER NOT NULL,
        total REAL NOT NULL,
        FOREIGN KEY (user_id) REFERENCES users(id)
    );
''')

# Insert sample data
cur.executemany('INSERT INTO users (name) VALUES (?)', 
                [('Ana',), ('Bob',), ('Carol',)])
cur.executemany('INSERT INTO orders (user_id, total) VALUES (?, ?)', [
    (1, 19.99), (1, 5.00), (1, 25.50),  # Ana's orders
    (2, 12.50), (2, 45.00),              # Bob's orders
    # Note: Carol has no orders
])
conn.commit()

# ═══════════════════════════════════════════════════════════════════
# 📝 YOUR CODE HERE: Write a query to get total spend per user
# ═══════════════════════════════════════════════════════════════════

# TODO: Replace 'pass' with your query
# Hint: Use LEFT JOIN, GROUP BY, SUM(), and COALESCE for NULL handling

query = '''
    -- Your SQL query here
    SELECT 
        u.name,
        -- Add more columns...
    FROM users u
    -- Add JOIN clause...
    -- Add GROUP BY clause...
    -- Add ORDER BY clause...
'''

# Uncomment to run your query:
# cur.execute(query)
# for row in cur.fetchall():
#     print(row)

# ═══════════════════════════════════════════════════════════════════
# 💡 SOLUTION (run after trying yourself):
# ═══════════════════════════════════════════════════════════════════

solution_query = '''
    SELECT 
        u.name,
        COUNT(o.id) as order_count,
        COALESCE(SUM(o.total), 0) as total_spend
    FROM users u
    LEFT JOIN orders o ON u.id = o.user_id
    GROUP BY u.id
    ORDER BY total_spend DESC
'''

print("✅ Solution Results:")
print("-" * 40)
cur.execute(solution_query)
for row in cur.fetchall():
    print(f"  {row['name']}: {row['order_count']} orders, ${row['total_spend']:.2f}")

conn.close()

---

## 🔒 Activity 2: Transactions and ACID Properties

```
╔══════════════════════════════════════════════════════════════════╗
║  Understanding Transactions                                      ║
║                                                                  ║
║  ACID Properties:                                                ║
║  • Atomicity   - All or nothing                                 ║
║  • Consistency - Valid state before and after                   ║
║  • Isolation   - Concurrent transactions don't interfere        ║
║  • Durability  - Committed changes persist                      ║
╚══════════════════════════════════════════════════════════════════╝
```

### Why Transactions Matter:

Consider a bank transfer:
1. Deduct $100 from Account A
2. Add $100 to Account B

What if step 1 succeeds but step 2 fails? 💸 Money disappears!

**Transactions ensure both steps succeed or both fail.**

### SQL Transaction Commands:

| Command | Description |
|---------|-------------|
| `BEGIN` | Start a transaction |
| `COMMIT` | Save all changes |
| `ROLLBACK` | Undo all changes since BEGIN |

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🚫 BAD PRACTICE: No Transaction Safety                          ║
# ╚══════════════════════════════════════════════════════════════════╝
# What NOT to do when multiple operations need to succeed together

import sqlite3

conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# Create accounts table
cur.execute('''
    CREATE TABLE accounts (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        balance REAL NOT NULL
    )
''')
cur.executemany('INSERT INTO accounts (name, balance) VALUES (?, ?)', [
    ('Alice', 1000.00),
    ('Bob', 500.00)
])
conn.commit()

# ❌ Bad: No transaction protection
# What if the program crashes between these operations?
print("❌ BAD: No Transaction Protection")
print("Before transfer:")
cur.execute('SELECT * FROM accounts')
for row in cur.fetchall():
    print(f"  {row}")

# Simulate transfer: Alice -> Bob $100
cur.execute('UPDATE accounts SET balance = balance - 100 WHERE name = ?', ('Alice',))
# 💥 CRASH HAPPENS HERE! (simulated)
# Money is gone from Alice, but never added to Bob!
# cur.execute('UPDATE accounts SET balance = balance + 100 WHERE name = ?', ('Bob',))

print("\n⚠️ After partial failure (simulated crash):")
cur.execute('SELECT * FROM accounts')
for row in cur.fetchall():
    print(f"  {row}")
print("  🚨 $100 disappeared!")

conn.close()

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🌱 NOVICE: Basic Transactions                                   ║
# ╚══════════════════════════════════════════════════════════════════╝
# Using BEGIN, COMMIT, and ROLLBACK

import sqlite3

conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# Create accounts table
cur.execute('''
    CREATE TABLE accounts (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        balance REAL NOT NULL
    )
''')
cur.executemany('INSERT INTO accounts (name, balance) VALUES (?, ?)', [
    ('Alice', 1000.00),
    ('Bob', 500.00)
])
conn.commit()

print("🌱 NOVICE: Basic Transaction Example\n")

# Show initial state
print("Initial balances:")
cur.execute('SELECT name, balance FROM accounts')
for row in cur.fetchall():
    print(f"  {row[0]}: ${row[1]:.2f}")

# ═══════════════════════════════════════════════════════════════════
# Example 1: Successful Transaction with COMMIT
# ═══════════════════════════════════════════════════════════════════

print("\n📝 Example 1: Successful Transfer (Alice -> Bob $100)")

cur.execute('BEGIN')  # Start transaction
try:
    cur.execute('UPDATE accounts SET balance = balance - 100 WHERE name = ?', ('Alice',))
    cur.execute('UPDATE accounts SET balance = balance + 100 WHERE name = ?', ('Bob',))
    cur.execute('COMMIT')  # Save changes
    print("  ✅ Transaction committed!")
except Exception as e:
    cur.execute('ROLLBACK')  # Undo changes
    print(f"  ❌ Transaction rolled back: {e}")

cur.execute('SELECT name, balance FROM accounts')
for row in cur.fetchall():
    print(f"  {row[0]}: ${row[1]:.2f}")

# ═══════════════════════════════════════════════════════════════════
# Example 2: Failed Transaction with ROLLBACK
# ═══════════════════════════════════════════════════════════════════

print("\n📝 Example 2: Failed Transfer (ROLLBACK demonstration)")

cur.execute('BEGIN')
cur.execute('UPDATE accounts SET balance = balance * 1.1')  # 10% increase
print("  After update (before rollback):")
cur.execute('SELECT name, balance FROM accounts')
for row in cur.fetchall():
    print(f"    {row[0]}: ${row[1]:.2f}")

# Oops! We changed our mind - ROLLBACK!
cur.execute('ROLLBACK')
print("  After ROLLBACK:")
cur.execute('SELECT name, balance FROM accounts')
for row in cur.fetchall():
    print(f"    {row[0]}: ${row[1]:.2f}")

conn.close()

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🔧 INTERMEDIATE: Context Manager Transactions                   ║
# ╚══════════════════════════════════════════════════════════════════╝
# Using Python's context manager for automatic transaction handling

import sqlite3
from contextlib import contextmanager

def create_banking_db() -> sqlite3.Connection:
    """Create and populate a banking database."""
    conn = sqlite3.connect(':memory:')
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()
    
    cur.execute('''
        CREATE TABLE accounts (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            balance REAL NOT NULL CHECK(balance >= 0)
        )
    ''')
    cur.executemany('INSERT INTO accounts (name, balance) VALUES (?, ?)', [
        ('Alice', 1000.00),
        ('Bob', 500.00),
        ('Charlie', 250.00)
    ])
    conn.commit()
    return conn

@contextmanager
def transaction(conn: sqlite3.Connection):
    """
    Context manager for database transactions.
    Automatically commits on success, rolls back on failure.
    """
    try:
        yield conn
        conn.commit()
    except Exception as e:
        conn.rollback()
        raise

def transfer_money(conn: sqlite3.Connection, from_name: str, to_name: str, amount: float) -> bool:
    """
    Transfer money between accounts with transaction safety.
    Returns True if successful, raises exception on failure.
    """
    with transaction(conn):
        cur = conn.cursor()
        
        # Check source has sufficient funds
        cur.execute('SELECT balance FROM accounts WHERE name = ?', (from_name,))
        row = cur.fetchone()
        if not row:
            raise ValueError(f"Account '{from_name}' not found")
        if row['balance'] < amount:
            raise ValueError(f"Insufficient funds: ${row['balance']:.2f} < ${amount:.2f}")
        
        # Perform transfer
        cur.execute('UPDATE accounts SET balance = balance - ? WHERE name = ?', (amount, from_name))
        cur.execute('UPDATE accounts SET balance = balance + ? WHERE name = ?', (amount, to_name))
        
        return True

def show_balances(conn: sqlite3.Connection):
    """Display all account balances."""
    cur = conn.cursor()
    cur.execute('SELECT name, balance FROM accounts ORDER BY name')
    for row in cur.fetchall():
        print(f"  {row['name']}: ${row['balance']:.2f}")

# ═══════════════════════════════════════════════════════════════════
# 🔧 Demonstration
# ═══════════════════════════════════════════════════════════════════

conn = create_banking_db()

print("🔧 INTERMEDIATE: Context Manager Transactions\n")

print("Initial balances:")
show_balances(conn)

# Successful transfer
print("\n📝 Transfer $200 from Alice to Bob:")
try:
    transfer_money(conn, 'Alice', 'Bob', 200.00)
    print("  ✅ Transfer successful!")
except ValueError as e:
    print(f"  ❌ Transfer failed: {e}")

show_balances(conn)

# Failed transfer (insufficient funds)
print("\n📝 Attempt to transfer $1000 from Charlie to Alice:")
try:
    transfer_money(conn, 'Charlie', 'Alice', 1000.00)
    print("  ✅ Transfer successful!")
except ValueError as e:
    print(f"  ❌ Transfer failed: {e}")

print("\nBalances unchanged due to rollback:")
show_balances(conn)

conn.close()

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  ✨ PYTHONIC: Enterprise Transaction Patterns                    ║
# ╚══════════════════════════════════════════════════════════════════╝
# Production-ready transaction handling with retry logic and logging

import sqlite3
from typing import Optional, Callable, TypeVar, List
from dataclasses import dataclass
from contextlib import contextmanager
from functools import wraps
import time

T = TypeVar('T')

# ═══════════════════════════════════════════════════════════════════
# ✨ Domain Models
# ═══════════════════════════════════════════════════════════════════

@dataclass
class Account:
    """Bank account entity."""
    id: Optional[int]
    name: str
    balance: float
    
    def __post_init__(self):
        if self.balance < 0:
            raise ValueError("Balance cannot be negative")

@dataclass
class TransferResult:
    """Result of a money transfer operation."""
    success: bool
    from_account: str
    to_account: str
    amount: float
    message: str
    new_from_balance: Optional[float] = None
    new_to_balance: Optional[float] = None

# ═══════════════════════════════════════════════════════════════════
# ✨ Banking Service with Transaction Management
# ═══════════════════════════════════════════════════════════════════

class BankingService:
    """
    Enterprise banking service with ACID-compliant transactions.
    
    Features:
    - Automatic retry on deadlock
    - Audit logging
    - Balance validation
    - Batch operations
    """
    
    def __init__(self, db_path: str = ':memory:'):
        self.db_path = db_path
        self._init_database()
    
    def _init_database(self) -> None:
        """Initialize database schema."""
        with self._get_connection() as conn:
            conn.executescript('''
                CREATE TABLE IF NOT EXISTS accounts (
                    id INTEGER PRIMARY KEY,
                    name TEXT UNIQUE NOT NULL,
                    balance REAL NOT NULL CHECK(balance >= 0)
                );
                
                CREATE TABLE IF NOT EXISTS transactions_log (
                    id INTEGER PRIMARY KEY,
                    timestamp TEXT DEFAULT CURRENT_TIMESTAMP,
                    from_account TEXT,
                    to_account TEXT,
                    amount REAL,
                    status TEXT,
                    message TEXT
                );
            ''')
    
    @contextmanager
    def _get_connection(self):
        """Get database connection with proper configuration."""
        conn = sqlite3.connect(self.db_path)
        conn.row_factory = sqlite3.Row
        conn.execute('PRAGMA foreign_keys = ON')
        try:
            yield conn
        finally:
            conn.close()
    
    @contextmanager
    def _transaction(self, conn: sqlite3.Connection):
        """Transaction context manager with automatic commit/rollback."""
        try:
            yield
            conn.commit()
        except Exception:
            conn.rollback()
            raise
    
    def create_account(self, name: str, initial_balance: float = 0.0) -> Account:
        """Create a new account."""
        with self._get_connection() as conn:
            with self._transaction(conn):
                cursor = conn.execute(
                    'INSERT INTO accounts (name, balance) VALUES (?, ?)',
                    (name, initial_balance)
                )
                return Account(cursor.lastrowid, name, initial_balance)
    
    def get_balance(self, name: str) -> Optional[float]:
        """Get account balance."""
        with self._get_connection() as conn:
            cursor = conn.execute(
                'SELECT balance FROM accounts WHERE name = ?', (name,)
            )
            row = cursor.fetchone()
            return row['balance'] if row else None
    
    def transfer(self, from_name: str, to_name: str, amount: float) -> TransferResult:
        """
        Transfer money between accounts with full ACID compliance.
        Logs all transactions for audit purposes.
        """
        if amount <= 0:
            return TransferResult(False, from_name, to_name, amount, 
                                 "Amount must be positive")
        
        with self._get_connection() as conn:
            try:
                with self._transaction(conn):
                    cur = conn.cursor()
                    
                    # Lock rows in consistent order to prevent deadlocks
                    accounts = sorted([from_name, to_name])
                    for name in accounts:
                        cur.execute(
                            'SELECT balance FROM accounts WHERE name = ? FOR UPDATE',
                            (name,)
                        ) if hasattr(cur, 'for_update') else cur.execute(
                            'SELECT balance FROM accounts WHERE name = ?',
                            (name,)
                        )
                    
                    # Check source balance
                    cur.execute('SELECT balance FROM accounts WHERE name = ?', 
                               (from_name,))
                    row = cur.fetchone()
                    if not row:
                        raise ValueError(f"Account '{from_name}' not found")
                    if row['balance'] < amount:
                        raise ValueError(f"Insufficient funds")
                    
                    # Perform transfer
                    cur.execute(
                        'UPDATE accounts SET balance = balance - ? WHERE name = ?',
                        (amount, from_name)
                    )
                    cur.execute(
                        'UPDATE accounts SET balance = balance + ? WHERE name = ?',
                        (amount, to_name)
                    )
                    
                    # Get new balances
                    cur.execute('SELECT balance FROM accounts WHERE name = ?', 
                               (from_name,))
                    new_from = cur.fetchone()['balance']
                    cur.execute('SELECT balance FROM accounts WHERE name = ?', 
                               (to_name,))
                    new_to = cur.fetchone()['balance']
                    
                    # Log successful transaction
                    cur.execute('''
                        INSERT INTO transactions_log 
                        (from_account, to_account, amount, status, message)
                        VALUES (?, ?, ?, 'SUCCESS', 'Transfer completed')
                    ''', (from_name, to_name, amount))
                    
                    return TransferResult(
                        True, from_name, to_name, amount,
                        "Transfer completed successfully",
                        new_from, new_to
                    )
                    
            except ValueError as e:
                # Log failed transaction
                with self._transaction(conn):
                    conn.execute('''
                        INSERT INTO transactions_log 
                        (from_account, to_account, amount, status, message)
                        VALUES (?, ?, ?, 'FAILED', ?)
                    ''', (from_name, to_name, amount, str(e)))
                
                return TransferResult(False, from_name, to_name, amount, str(e))
    
    def batch_transfer(self, transfers: List[tuple]) -> List[TransferResult]:
        """
        Perform multiple transfers atomically.
        All succeed or all fail.
        """
        results = []
        with self._get_connection() as conn:
            try:
                with self._transaction(conn):
                    for from_name, to_name, amount in transfers:
                        cur = conn.cursor()
                        
                        # Validate and execute each transfer
                        cur.execute('SELECT balance FROM accounts WHERE name = ?', 
                                   (from_name,))
                        row = cur.fetchone()
                        if not row or row['balance'] < amount:
                            raise ValueError(
                                f"Cannot transfer ${amount:.2f} from {from_name}"
                            )
                        
                        cur.execute(
                            'UPDATE accounts SET balance = balance - ? WHERE name = ?',
                            (amount, from_name)
                        )
                        cur.execute(
                            'UPDATE accounts SET balance = balance + ? WHERE name = ?',
                            (amount, to_name)
                        )
                        
                        results.append(TransferResult(
                            True, from_name, to_name, amount, "Pending commit"
                        ))
                    
                    # Update all results to success after commit
                    for r in results:
                        r.message = "Batch transfer successful"
                        
            except Exception as e:
                return [TransferResult(False, t[0], t[1], t[2], 
                                      f"Batch failed: {e}") for t in transfers]
        
        return results
    
    def get_transaction_history(self, limit: int = 10) -> List[dict]:
        """Get recent transaction history."""
        with self._get_connection() as conn:
            cursor = conn.execute('''
                SELECT * FROM transactions_log 
                ORDER BY timestamp DESC LIMIT ?
            ''', (limit,))
            return [dict(row) for row in cursor.fetchall()]

# ═══════════════════════════════════════════════════════════════════
# ✨ Demonstration
# ═══════════════════════════════════════════════════════════════════

print("✨ PYTHONIC: Enterprise Banking Service\n")

bank = BankingService()

# Create accounts
for name, balance in [('Alice', 1000.0), ('Bob', 500.0), ('Charlie', 250.0)]:
    bank.create_account(name, balance)

print("📊 Initial Balances:")
for name in ['Alice', 'Bob', 'Charlie']:
    print(f"  {name}: ${bank.get_balance(name):.2f}")

# Single transfer
print("\n💸 Single Transfer: Alice -> Bob $200")
result = bank.transfer('Alice', 'Bob', 200.0)
print(f"  Status: {'✅' if result.success else '❌'} {result.message}")
if result.success:
    print(f"  Alice: ${result.new_from_balance:.2f}, Bob: ${result.new_to_balance:.2f}")

# Failed transfer
print("\n💸 Failed Transfer: Charlie -> Alice $1000 (insufficient funds)")
result = bank.transfer('Charlie', 'Alice', 1000.0)
print(f"  Status: {'✅' if result.success else '❌'} {result.message}")

# Batch transfer
print("\n💸 Batch Transfer:")
batch_results = bank.batch_transfer([
    ('Alice', 'Charlie', 50.0),
    ('Bob', 'Charlie', 50.0),
])
for r in batch_results:
    print(f"  {r.from_account} -> {r.to_account}: ${r.amount:.2f} - {r.message}")

print("\n📊 Final Balances:")
for name in ['Alice', 'Bob', 'Charlie']:
    print(f"  {name}: ${bank.get_balance(name):.2f}")

print("\n📜 Transaction History:")
for txn in bank.get_transaction_history(5):
    print(f"  {txn['timestamp']}: {txn['from_account']} -> {txn['to_account']} "
          f"${txn['amount']:.2f} [{txn['status']}]")

---

## 🎯 Group Activity 2: Transaction Challenge

### Your Task:

Create a transaction that performs **multiple related operations** and properly handles success/failure.

**Scenario:** Process a "payday" where:
1. All employees get a 5% bonus from the company account
2. If any operation fails, ALL changes must be rolled back

### Requirements:
1. Create a company account with $10,000
2. Create 3 employee accounts with $0 each
3. In ONE transaction:
   - Calculate 5% bonus for each employee ($100 each from company)
   - Deduct from company, add to each employee
4. If company doesn't have enough funds, rollback everything

### 💡 Hints:
- Use `BEGIN` to start the transaction
- Check company balance before operations
- Use `COMMIT` on success, `ROLLBACK` on failure
- Consider what happens if one employee transfer fails

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  👩‍💻 YOUR WORKSPACE: Transaction Challenge                        ║
# ╚══════════════════════════════════════════════════════════════════╝
# Create a payday transaction that pays bonuses to all employees

import sqlite3

# Setup database
conn = sqlite3.connect(':memory:')
conn.row_factory = sqlite3.Row
cur = conn.cursor()

cur.execute('''
    CREATE TABLE accounts (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        balance REAL NOT NULL CHECK(balance >= 0)
    )
''')

# Create accounts: Company + 3 employees
cur.execute('INSERT INTO accounts (name, balance) VALUES (?, ?)', ('Company', 10000.00))
for emp in ['Employee1', 'Employee2', 'Employee3']:
    cur.execute('INSERT INTO accounts (name, balance) VALUES (?, ?)', (emp, 0.00))
conn.commit()

print("Initial balances:")
cur.execute('SELECT name, balance FROM accounts')
for row in cur.fetchall():
    print(f"  {row['name']}: ${row['balance']:.2f}")

# ═══════════════════════════════════════════════════════════════════
# 📝 YOUR CODE HERE: Implement the payday transaction
# ═══════════════════════════════════════════════════════════════════

bonus_per_employee = 100.00
employees = ['Employee1', 'Employee2', 'Employee3']
total_bonus = bonus_per_employee * len(employees)

# TODO: Implement transaction
# 1. Check if company has enough funds
# 2. Begin transaction
# 3. Deduct total bonus from company
# 4. Add bonus to each employee
# 5. Commit on success, rollback on failure

# Your code here:
# cur.execute('BEGIN')
# ...

# ═══════════════════════════════════════════════════════════════════
# 💡 SOLUTION (run after trying yourself):
# ═══════════════════════════════════════════════════════════════════

print("\n" + "=" * 50)
print("💡 SOLUTION:")
print("=" * 50)

# Check company balance
cur.execute('SELECT balance FROM accounts WHERE name = ?', ('Company',))
company_balance = cur.fetchone()['balance']

if company_balance >= total_bonus:
    try:
        cur.execute('BEGIN')
        
        # Deduct from company
        cur.execute(
            'UPDATE accounts SET balance = balance - ? WHERE name = ?',
            (total_bonus, 'Company')
        )
        
        # Add to each employee
        for emp in employees:
            cur.execute(
                'UPDATE accounts SET balance = balance + ? WHERE name = ?',
                (bonus_per_employee, emp)
            )
        
        cur.execute('COMMIT')
        print("✅ Payday transaction successful!")
        
    except Exception as e:
        cur.execute('ROLLBACK')
        print(f"❌ Transaction failed: {e}")
else:
    print(f"❌ Insufficient funds: ${company_balance:.2f} < ${total_bonus:.2f}")

# Show final balances
print("\nFinal balances:")
cur.execute('SELECT name, balance FROM accounts')
for row in cur.fetchall():
    print(f"  {row['name']}: ${row['balance']:.2f}")

conn.close()

---

## 🤝 Collaboration Guide

### Team Roles:

| Role | Responsibilities |
|------|------------------|
| 👷 **Schema Designer** | Define table structures, constraints, relationships |
| ✍️ **Query Author** | Write SQL queries, JOINs, aggregations |
| ✅ **Verifier** | Test queries, validate results, catch edge cases |

### Rotation Schedule:
- **Activity 1:** Rotate roles after completing the total spend query
- **Activity 2:** Rotate roles after completing the transaction challenge

### Discussion Topics:

1. **Schema Design:**
   - When should you use foreign keys?
   - What constraints should you add?
   - How do you balance normalization vs. query performance?

2. **JOINs:**
   - INNER vs LEFT JOIN - when to use each?
   - How do NULLs affect aggregations?
   - What happens with many-to-many relationships?

3. **Transactions:**
   - Real-world scenarios requiring transactions
   - What could go wrong without transactions?
   - How do you test transaction rollback?

---

## 📚 Session Recap

### What We Learned Today:

| Topic | Key Concept | Example |
|-------|-------------|---------|
| **JOINs** | Connect related tables | `SELECT ... FROM a JOIN b ON a.id = b.a_id` |
| **Foreign Keys** | Enforce relationships | `FOREIGN KEY (user_id) REFERENCES users(id)` |
| **GROUP BY** | Aggregate by category | `GROUP BY user_id` |
| **Aggregations** | Summarize data | `COUNT()`, `SUM()`, `AVG()` |
| **HAVING** | Filter aggregated results | `HAVING SUM(total) > 100` |
| **Transactions** | Atomic operations | `BEGIN ... COMMIT/ROLLBACK` |

### The Evolution of Our Database Code:

| Level | Characteristics |
|-------|-----------------|
| 🚫 **Bad** | No FK constraints, string formatting, no transactions |
| 🌱 **Novice** | Foreign keys enabled, BEGIN/COMMIT/ROLLBACK |
| 🔧 **Intermediate** | Context managers, validation, error handling |
| ✨ **Pythonic** | Service classes, dataclasses, audit logging, batch operations |

### SQL Quick Reference:

```sql
-- JOINs
SELECT u.name, o.total
FROM users u
INNER JOIN orders o ON u.id = o.user_id;  -- Only matching

SELECT u.name, o.total
FROM users u
LEFT JOIN orders o ON u.id = o.user_id;   -- All users

-- Aggregations
SELECT user_id, COUNT(*), SUM(total), AVG(total)
FROM orders
GROUP BY user_id
HAVING SUM(total) > 100;

-- Transactions
BEGIN;
UPDATE accounts SET balance = balance - 100 WHERE id = 1;
UPDATE accounts SET balance = balance + 100 WHERE id = 2;
COMMIT;  -- or ROLLBACK;
```

### 🎯 Key Takeaways:

1. **Always enable foreign keys** in SQLite (`PRAGMA foreign_keys = ON`)
2. **Use LEFT JOIN** when you need all records from one table
3. **COALESCE** handles NULL values in aggregations
4. **Transactions** ensure data integrity for related operations
5. **Context managers** automatically handle commit/rollback

### 🔜 Next Steps:
- Advanced SQL (subqueries, window functions)
- Database design patterns
- ORM libraries (SQLAlchemy)
- Building a complete data application